Мета:

Навчити (finetune) попередньо треновану велику мовну модель (LLM) на конкретному наборі даних для досягнення високої точності у вирішенні конкретної задачі, а також провести оцінку її ефективності.

Кроки для виконання завдання:

1. Підготовка середовища:
  * Встановіть необхідні бібліотеки: Hugging Face Transformers, TensorFlow або PyTorch, numpy, pandas.
  * Визначте та підготуйте доступ до обчислювальних ресурсів (локальний комп’ютер або сервер із GPU).
2. Вибір попередньо навченої LLM та набору даних:
  * Оберіть відповідну LLM, таку як Llama, Gemma, GPT-2, BERT, T5, або іншу модель із бібліотеки Hugging Face.
  * Завантажте або підготуйте набір даних.
  * Переконайтесь, що дані добре структуровані та підготовлені для навчання (наприклад, файли CSV або JSON).
3. Попередня обробка даних:
  * Виконайте попередню обробку тексту, включаючи токенізацію, видалення зайвих символів або перетворення в потрібний формат.
  * Використайте вбудовані токенізатори з бібліотеки Transformers для обраної моделі.
4. Налаштування (finetuning) моделі:
  * Завантажте обрану попередньо навчану модель.
  * Визначте гіперпараметри для навчання: розмір батчу, кількість епох, коефіцієнт навчання.
  * Виконайте налаштування моделі на вашому наборі даних за допомогою API бібліотеки Transformers (Hugging Face).
5. Оцінка ефективності моделі:
  * Розділіть ваш набір даних на навчальну та тестову (або валідаційну) вибірки.
  * Після навчання обчисліть показники ефективності, BLEU та ROUGE для задач генерації тексту.
  * Проаналізуйте результати та порівняйте їх із базовою (до навчання) продуктивністю моделі.
6. Оптимізація та повторне навчання:
  * На основі отриманих результатів, за потреби, налаштуйте гіперпараметри або передобробку даних.
  * Повторіть навчання моделі та порівняйте результати після оптимізації.
7. Документування результатів:
  * Зробіть висновки щодо ефективності моделі після налаштування.
  * Опишіть, які покращення були досягнуті та в яких випадках модель показала найкращі результати.
  * За можливості, надайте візуалізацію метрик для кращого розуміння результатів.

Мінімальні вимоги:

* Налаштувати (finetune) попередньо навчений LLM на конкретному наборі даних.
* Оцінити ефективність моделі за допомогою відповідних метрик.

Формат виконання: 

.ipynb блокнот код в .py з інференсом для запуску моделі та короткою документацією.

In [ ]:
import os
from huggingface_hub import hf_hub_download
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering
import torch
from torch.utils.data import Dataset

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased-distilled-squad')
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased-distilled-squad')


question_answerer = pipeline("question-answering", device='mps',
                              model=model, tokenizer=tokenizer)

In [3]:
# Add specific context instead of open-ended question
context = "Apache Kafka competitors include RabbitMQ, Amazon SQS, ActiveMQ, and Redis."
question = "What are competitors to Apache Kafka?"

result=question_answerer(question="What is a good example of a question answering dataset?",     context=context)
display(result)

{'score': 0.27250373363494873,
 'start': 33,
 'end': 54,
 'answer': 'RabbitMQ, Amazon SQS,'}

In [ ]:
import pandas as pd
import re
from datetime import datetime

def process_whatsapp_chat(file_path):
    pattern = r'\[(.*?)\s(.*?)\]\s(.*?):\s(.+)'
    datetimes, authors, messages = [], [], []
    
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.readlines()
            
    for line in content:
        line=line.replace('‎', '')
        match = re.match(pattern, line)
        if match:
            # Combine date and time into datetime object
            datetime_str = f"{match.group(1)} {match.group(2)}"
            datetime_obj = datetime.strptime(datetime_str, '%d.%m.%Y, %H:%M:%S')
            if 'image omitted' in match.group(4):
                continue
            if 'audio omitted' in match.group(4):
                continue
                                    
            datetimes.append(datetime_obj)
            authors.append(match.group(3))
            messages.append(match.group(4))
        else:
            if messages:
                messages[-1] += '\n' + line.strip()
    
    return pd.DataFrame({
        'Datetime': datetimes,
        'Author': authors,
        'Message': messages
    })

def merge_consecutive_messages(df):
    # Create a copy to avoid modifying original
    result = []
    i = 0
    
    while i < len(df):
        current_row = df.iloc[i].copy()
        merged_message = current_row['Message']
        
        # Look ahead for consecutive messages
        while (i + 1 < len(df) and 
               df.iloc[i + 1]['Author'] == current_row['Author']):
            merged_message += '\n' + df.iloc[i + 1]['Message']
            current_row['Datetime'] = df.iloc[i + 1]['Datetime']  # Take latest time
            i += 1
            
        current_row['Message'] = merged_message
        result.append(current_row)
        i += 1
    
    return pd.DataFrame(result)

# Usage
chat_file = 'data/_chat.txt'
df = process_whatsapp_chat(chat_file)
df = merge_consecutive_messages(df)

messages_by_author = (df.groupby(['Author', df.groupby('Author').cumcount()])
                     .Message
                     .first()
                     .unstack())

messages_by_author=messages_by_author.T[['Viktoriia ','Volodymyr']]
messages_by_author


In [ ]:
messages_by_author = messages_by_author.map(str)
# Convert df_merge to the required format
dataset = messages_by_author.rename(columns={
    'Viktoriia': 'question',
    'Volodymyr': 'answer'  # Assuming 'summary' is your target column
}).to_dict('records')

context =''
for i in dataset:
    context += i['question'] + ' ' + str(i['answer'])

context

'Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them. KICRF\'24 one pager UA final_1.0.pdf • 2 pages document omitted\nти тут виступайеш) нас запросилиЩооо??? 😁😁\nМожна деталі, будь ласка?\nДобре, виступаю)) тільки треба деталі)) можна тебе набрати?Соррі, немесил)) давай вже зранку? Щойно побачила просто(( та вранці) я уже сплю)\nуже с зала вийшов, набирай як сможещБуду готова дзвонити за кілька хвилин\nНема варіантів на жаль, тому ти сам цього разу. Наступного я обіцяю бути.\n\nТут варіанти або квитки за 550-650 баксів, і то без потяга і без готелів, що разом вийде під штуку, або ти сам.\nТому я пропоную зараз гроші не витрачати, допоможу тобі підготувати спіч деку ок\nвипадково🙂\ndo you have another Vika-partner? 🙂 only you…….\nв мене була співробітниця, але то була Вікуся)😁😁 а чо питайеш?😂😂\nТа жартую))\nВсім написала.\nЩе по відео - що там Ліна?) полігон називається Прибан, він в Сторожинецькому районі, Чернівецько

In [56]:
# Convert dataset format
qa_dataset = []
for item in dataset:  # your original dataset
    # print(context.find(item['answer']))
    qa_dataset.append({
        'context': context,
        'question': item['question'],  # You can modify this based on your needs
        'answer': {
            'text': item['answer'],
            'answer_start': int(context.find(item['answer']))  # Find start position of answer in text
        }
    })


In [57]:
class QADataset(Dataset):
    def __init__(self, data, tokenizer, max_length=384):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        
        # Get answer positions before tokenization
        answer_text = item['answer']['text']
        answer_start = item['answer']['answer_start']
        
        # Tokenize
        encoding = self.tokenizer(
            item['question'],
            item['context'],
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )

        # Calculate start and end tokens by tokenizing segments
        context_before_answer = item['context'][:answer_start]
        answer = item['context'][answer_start:answer_start + len(answer_text)]
        
        start_tokens = len(self.tokenizer.encode(item['question'])) + len(self.tokenizer.encode(context_before_answer)) - 2  # -2 for [CLS] tokens
        end_tokens = start_tokens + len(self.tokenizer.encode(answer)) - 2  # -2 for [CLS] tokens

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'start_positions': torch.tensor(start_tokens, dtype=torch.long),
            'end_positions': torch.tensor(end_tokens - 1, dtype=torch.long)  # -1 because end position should be the last token of answer
        }


In [58]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

train_dataset = QADataset(qa_dataset, tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)


In [59]:
# Fine-tune the model
trainer.train()


  0%|          | 0/585 [00:00<?, ?it/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Token indices sequence length is longer than the specified maximum sequence length for this model (270570 > 512). Running this sequence through the model will result in indexing errors
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation st

KeyboardInterrupt: 

Нажаль в мене не вистачає потужності, щоб дотренувати цю модель